# SOFTKILL-9000: Multi-Agent Motion Capture Simulation

This notebook demonstrates how to run SOFTKILL-9000 in Google Colab.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BkAsDrP/Softkill9000/blob/main/examples/run_in_colab.ipynb)

---

## Overview

SOFTKILL-9000 is a multi-agent reinforcement learning system for motion capture simulation. This notebook will:

1. Install the package from GitHub
2. Run a basic simulation
3. Visualize agent trajectories
4. Demonstrate Q-learning training
5. Show advanced customization options

## 1. Installation

Install SOFTKILL-9000 directly from the GitHub repository:

In [ ]:
# Install the package from GitHub
!pip install git+https://github.com/BkAsDrP/Softkill9000.git -q

# Verify installation
import softkill9000
print(f"✅ SOFTKILL-9000 v{softkill9000.__version__} installed successfully!")

In [ ]:
# Upgrade NumPy to resolve dependency conflicts
# Colab comes with NumPy 1.x, but softkill9000 needs NumPy 2.x for modern ML libraries
# (jax, opencv, pytensor, thinc all require numpy>=2.0)
!pip install --upgrade "numpy>=2.0.0,<3.0.0" -q

# Verify NumPy version
import numpy as np
print(f"✅ NumPy {np.__version__} ready (compatible with JAX, OpenCV, PyTensor)")

## 2. Basic Simulation

Let's run a basic simulation with default configuration:

In [ ]:
from softkill9000.simulator import MissionSimulator
from softkill9000.agents.agent import Agent
from softkill9000.environments.environment import CosmicScenario
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Create agents
agents = [
    Agent(agent_id="Longsight", role="scout"),
    Agent(agent_id="Lifebinder", role="medic"),
    Agent(agent_id="Specter", role="assault")
]

# Create scenario
scenario = CosmicScenario.generate_random(
    width=100.0,
    height=100.0,
    num_objectives=3,
    num_obstacles=5
)

# Initialize simulator
simulator = MissionSimulator(
    agents=agents,
    scenario=scenario,
    timesteps=20
)

# Run simulation
print("🚀 Starting simulation...\n")
results = simulator.run_simulation()

# Display results
print("\n" + "="*60)
print("📊 SIMULATION RESULTS")
print("="*60)
print(f"Total Reward: {results['total_reward']:.2f}")
print(f"Objectives Completed: {results['objectives_completed']}")
print(f"Average Health: {results['average_health']:.1f}%")
print(f"Final Morale: {results['final_morale']:.1f}%")
print(f"Completion Time: {results['completion_time']:.2f}s")
print("="*60)

## 3. Visualize Agent Trajectories

Plot the paths taken by each agent during the mission:

In [ ]:
import matplotlib.pyplot as plt
from softkill9000.visualization import plot_agent_trajectories

# Plot trajectories
fig = plot_agent_trajectories(results['agent_histories'], scenario)
plt.show()

print("\n📍 Agent Movement Summary:")
for agent_id, history in results['agent_histories'].items():
    positions = [(h['position']['x'], h['position']['y']) for h in history]
    start = positions[0]
    end = positions[-1]
    distance = sum(
        np.sqrt((positions[i+1][0] - positions[i][0])**2 + 
                (positions[i+1][1] - positions[i][1])**2)
        for i in range(len(positions)-1)
    )
    print(f"  {agent_id}: {distance:.1f} units traveled")

## 4. Performance Metrics Over Time

Visualize how agent stats evolved during the simulation:

In [ ]:
from softkill9000.visualization import plot_performance_metrics

# Plot performance metrics
fig = plot_performance_metrics(results['agent_histories'])
plt.tight_layout()
plt.show()

print("\n📈 Performance Analysis:")
for agent_id, history in results['agent_histories'].items():
    initial_health = history[0]['health']
    final_health = history[-1]['health']
    health_change = final_health - initial_health
    
    initial_morale = history[0]['morale']
    final_morale = history[-1]['morale']
    morale_change = final_morale - initial_morale
    
    print(f"  {agent_id}:")
    print(f"    Health: {initial_health:.1f} → {final_health:.1f} ({health_change:+.1f})")
    print(f"    Morale: {initial_morale:.1f} → {final_morale:.1f} ({morale_change:+.1f})")

## 5. Q-Learning Training

Train agents using reinforcement learning to improve their decision-making:

In [ ]:
from softkill9000.environments.environment import QLearningTrainer
from softkill9000.config.models import QLearningConfig

# Configure Q-learning
qlearning_config = QLearningConfig(
    learning_rate=0.1,
    discount_factor=0.95,
    exploration_rate=0.2,
    min_exploration_rate=0.01,
    exploration_decay=0.995
)

# Create fresh agents for training
training_agents = [
    Agent(agent_id="Scout-1", role="scout"),
    Agent(agent_id="Medic-1", role="medic"),
    Agent(agent_id="Assault-1", role="assault")
]

# Initialize trainer
trainer = QLearningTrainer(config=qlearning_config)

# Training loop
print("🎓 Training agents with Q-Learning...\n")
episodes = 50
rewards_history = []

for episode in range(episodes):
    # Create new scenario for each episode
    training_scenario = CosmicScenario.generate_random(
        width=100.0,
        height=100.0,
        num_objectives=3,
        num_obstacles=5
    )
    
    # Run training episode
    sim = MissionSimulator(
        agents=training_agents,
        scenario=training_scenario,
        timesteps=20
    )
    
    results = sim.run_simulation()
    episode_reward = results['total_reward']
    rewards_history.append(episode_reward)
    
    # Train agents based on results
    for agent in training_agents:
        # Simplified training (in practice, you'd pass state-action-reward tuples)
        state = hash((agent.position.x, agent.position.y, agent.stats.health))
        action = agent.last_action if hasattr(agent, 'last_action') else 'move_forward'
        next_state = hash((agent.position.x + 1, agent.position.y, agent.stats.health))
        
        trainer.train_agent(
            agent=agent,
            state=state,
            action=action,
            reward=episode_reward / len(training_agents),
            next_state=next_state
        )
    
    if (episode + 1) % 10 == 0:
        avg_reward = np.mean(rewards_history[-10:])
        print(f"Episode {episode + 1}/{episodes} - Avg Reward (last 10): {avg_reward:.2f}")

# Plot training progress
plt.figure(figsize=(10, 6))
plt.plot(rewards_history, alpha=0.6, label='Episode Reward')
plt.plot(np.convolve(rewards_history, np.ones(10)/10, mode='valid'), 
         linewidth=2, label='Moving Average (10)')
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.title('Q-Learning Training Progress')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"\n✅ Training completed!")
print(f"Final exploration rate: {trainer.config.exploration_rate:.3f}")
print(f"Average reward (last 10 episodes): {np.mean(rewards_history[-10:]):.2f}")

## 6. Custom Configuration

Use YAML configuration for advanced customization:

In [ ]:
from softkill9000.config.models import load_config_from_yaml
import yaml

# Create custom configuration
custom_config = """
simulation:
  timesteps: 30
  random_seed: 12345
  enable_visualization: true
  log_level: INFO

agents:
  - agent_id: "Alpha"
    role: "scout"
    initial_health: 100.0
    initial_morale: 100.0
    speed_multiplier: 1.5
  
  - agent_id: "Bravo"
    role: "assault"
    initial_health: 120.0
    initial_morale: 90.0
    speed_multiplier: 1.0
  
  - agent_id: "Charlie"
    role: "medic"
    initial_health: 80.0
    initial_morale: 100.0
    speed_multiplier: 1.2

mission:
  width: 150.0
  height: 150.0
  num_objectives: 5
  num_obstacles: 8
  difficulty: "hard"

qlearning:
  learning_rate: 0.15
  discount_factor: 0.98
  exploration_rate: 0.3
  min_exploration_rate: 0.05
  exploration_decay: 0.99
"""

# Save to temporary file
with open('/tmp/custom_config.yaml', 'w') as f:
    f.write(custom_config)

# Load configuration
config = load_config_from_yaml('/tmp/custom_config.yaml')

print("⚙️  Custom Configuration Loaded:")
print(f"  Simulation timesteps: {config.simulation.timesteps}")
print(f"  Number of agents: {len(config.agents)}")
print(f"  Mission size: {config.mission.width}x{config.mission.height}")
print(f"  Difficulty: {config.mission.difficulty}")

# Create agents from config
custom_agents = [Agent(**agent.model_dump()) for agent in config.agents]

# Create scenario from config
custom_scenario = CosmicScenario.generate_random(
    width=config.mission.width,
    height=config.mission.height,
    num_objectives=config.mission.num_objectives,
    num_obstacles=config.mission.num_obstacles
)

# Run custom simulation
custom_sim = MissionSimulator(
    agents=custom_agents,
    scenario=custom_scenario,
    timesteps=config.simulation.timesteps
)

print("\n🚀 Running custom simulation...")
custom_results = custom_sim.run_simulation()

print("\n📊 Custom Simulation Results:")
print(f"  Total Reward: {custom_results['total_reward']:.2f}")
print(f"  Objectives: {custom_results['objectives_completed']}")
print(f"  Time: {custom_results['completion_time']:.2f}s")

## 7. Compare Multiple Scenarios

Run multiple simulations and compare results:

In [ ]:
# Run multiple scenarios
print("🔄 Running multiple scenarios...\n")

scenarios_data = []
num_runs = 10

for run in range(num_runs):
    # Create new agents and scenario for each run
    agents = [
        Agent(agent_id=f"Agent-{i}", role=["scout", "medic", "assault"][i % 3])
        for i in range(3)
    ]
    
    scenario = CosmicScenario.generate_random(
        width=100.0,
        height=100.0,
        num_objectives=3,
        num_obstacles=5
    )
    
    sim = MissionSimulator(agents=agents, scenario=scenario, timesteps=20)
    results = sim.run_simulation()
    
    scenarios_data.append({
        'run': run + 1,
        'reward': results['total_reward'],
        'objectives': results['objectives_completed'],
        'health': results['average_health'],
        'time': results['completion_time']
    })

# Analyze results
import pandas as pd

df = pd.DataFrame(scenarios_data)

print("📊 Statistical Summary:")
print(df.describe())

# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].bar(df['run'], df['reward'], color='steelblue')
axes[0, 0].axhline(df['reward'].mean(), color='red', linestyle='--', label='Mean')
axes[0, 0].set_xlabel('Run')
axes[0, 0].set_ylabel('Total Reward')
axes[0, 0].set_title('Reward by Scenario')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].bar(df['run'], df['objectives'], color='green')
axes[0, 1].axhline(df['objectives'].mean(), color='red', linestyle='--', label='Mean')
axes[0, 1].set_xlabel('Run')
axes[0, 1].set_ylabel('Objectives Completed')
axes[0, 1].set_title('Objectives by Scenario')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].bar(df['run'], df['health'], color='orange')
axes[1, 0].axhline(df['health'].mean(), color='red', linestyle='--', label='Mean')
axes[1, 0].set_xlabel('Run')
axes[1, 0].set_ylabel('Average Health (%)')
axes[1, 0].set_title('Final Health by Scenario')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].bar(df['run'], df['time'], color='purple')
axes[1, 1].axhline(df['time'].mean(), color='red', linestyle='--', label='Mean')
axes[1, 1].set_xlabel('Run')
axes[1, 1].set_ylabel('Time (seconds)')
axes[1, 1].set_title('Completion Time by Scenario')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✅ Completed {num_runs} scenario runs")
print(f"Average reward: {df['reward'].mean():.2f} ± {df['reward'].std():.2f}")

## 🎯 Next Steps

Explore more features:

1. **REST API**: Deploy the FastAPI server for remote access
2. **Custom Agents**: Create agents with specialized behaviors
3. **Advanced Scenarios**: Design complex mission environments
4. **Extended Training**: Train agents for more episodes to see performance improvements
5. **Visualization**: Create custom plots and animations

### Documentation

- [Architecture Guide](https://github.com/BkAsDrP/Softkill9000/blob/main/docs/architecture.md)
- [API Reference](https://github.com/BkAsDrP/Softkill9000/blob/main/docs/api_reference.md)
- [User Guide](https://github.com/BkAsDrP/Softkill9000/blob/main/docs/user_guide.md)
- [Deployment Guide](https://github.com/BkAsDrP/Softkill9000/blob/main/docs/deployment.md)

### Repository

- GitHub: [BkAsDrP/Softkill9000](https://github.com/BkAsDrP/Softkill9000)
- Issues: [Report bugs or request features](https://github.com/BkAsDrP/Softkill9000/issues)
- License: MIT

---

**Happy simulating! 🚀**